# Plotting forming limit curves from forming test data

This analysis starts with data exported from the GOM-correlate software.

Each experiment has 3 sections and each section has data for the major and minor strain. Each experiment is repeated multiple times with multiple sample geometries.

We plot 2 forming limit curves (FLC) from the data using two different stopping criteria:
* Formation of a crack in the sample
* First derivative necking 

Formation of a crack in the material is determined by inspection of the sample images.

First derivative necking determines the necking time to be when the strain in the sample becomes localised. This is determined numerically as the point at which the major strain in the necking region is 10 times greater than the major strain in the non-nekcing section.

In [ ]:
from pathlib import Path
import glob
from typing import List, Union, Tuple, Dict
from datetime import datetime

from scipy.signal import find_peaks
from scipy.ndimage import median_filter
import numpy as np
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import pandas as pd
from ipywidgets import fixed, interact, IntSlider, SelectMultiple, Checkbox
import dic_analysis.io

import utilities

The fracture time of each sample is determined by visual inspection of the sample images at each time step. The fracture times are listed here, one for each sample.

In [ ]:
fracture_times = {
    '10mm_001': 145,
    '10mm_002': 137,
    '10mm_003': 143,
    '20mm_001': 139,
    '20mm_002': 134,
    '20mm_003': 140,
    '40mm_001': 151,
    '40mm_002': 151,
    '40mm_003': 149,
    '60mm_001': 155,
    '60mm_002': 153,
    '60mm_003': 155,
    '120mm_001': 150,
    '120mm_002': 149,
    '120mm_003': 148,
    'fullcircle_001': 152,
    'fullcircle_002': 157,
    'fullcircle_003': 151,
}

sample_sizes = ["10mm", "20mm", "40mm", "60mm", "120mm", "fullcircle"]

# Download Data
The strain data dervied from the GOM Correlate software (~50 MB) are downloaded to `data/nakazima`.

In [ ]:
data_locations = utilities.read_data_yaml("../data/zenodo_URLs.yaml")["nakazima_tests"]
print(data_locations)

data_folder = utilities.get_file_from_url('../data/nakazima', unzip=True, **data_locations)
print(data_folder)

First we load the DIC data from the files up to the time of fracture for each sample.

In [ ]:
def get_time_from_file(path: Path) -> datetime:
    """Get the time of the measurement from the data file."""
    sample_time_string = pd.read_csv(path, skiprows=2, delimiter=";", nrows=1)["date"].values[0]
    return datetime.fromisoformat(sample_time_string)


def load_data(file_directory: str, fracture_time: int) -> Tuple[List[np.ndarray], List[float]]:
    """Load strain data from files. 
    
    Output is a list of numpy arrays, one for each timestep and a list of times, one for each timestep."""
    frames = []
    sample_times = []
    file_list = sorted(glob.glob(f"{file_directory}/*.csv"))
    
    # We ignore the first frame since it is before the experiment begins and we ignore the fracture frame and any after
    start_time = get_time_from_file(Path(file_list[1]))

    for index, file_name in enumerate(file_list):
        # Add one to index becuase python is zero based but file numbering is one based.
        if 1 < index + 1 < fracture_time:
            file_path = Path(file_name)
            sample_times.append((get_time_from_file(file_path) - start_time).total_seconds())
            data = np.genfromtxt(file_path, skip_header=6, delimiter=";", usecols=[5])
            frames.append(data)
    return frames, sample_times

major_strain_data = {}
minor_strain_data = {}
frame_times = {}
for experiment_name, fracture_time in tqdm(fracture_times.items()):
    file_directory = data_folder/ f"{experiment_name}/Section one/major strain"
    major_strain_data[experiment_name], time = load_data(file_directory, fracture_time)    
    frame_times[experiment_name] = time
    
    file_directory = data_folder / f"{experiment_name}/Section one/minor strain"
    minor_strain_data[experiment_name], _ = load_data(file_directory, fracture_time)

Make a results folder for figure outputs if it does not yet exist

In [ ]:
RESULTS_DIR = Path('../results')
RESULTS_DIR.mkdir(exist_ok=True, parents=True)

## 1). FLC from crack formation
The simplest way to define the stop points for the strain paths is to plot them up to the point of sample fracture. You can select or deselect experiments to be shown in the graph by holding the **Ctrl** key and clicking on the experiment name in the list.

In [ ]:
def get_color(name: str, sample_names: List[str]) -> tuple:
    """Returns a RGBA tuple for the dataset based on its label.""" 
    index = sample_names.index(name.split("_")[0])
    color_value = index / len(sample_names)
    color = plt.get_cmap("viridis")(color_value)
    return color

def get_marker(name: str) -> str:
    """Returns a symbol for the dataset based on the repeat number.""" 
    symbols = [".", "+", "x"] 
    repeat_number = int(name.split("_")[1])
    return symbols[repeat_number - 1]

def plot_strain_paths(names_to_plot: List[str], data: Dict[str, np.ndarray], title: str):
    plt.figure(figsize=(10,6))

    for experiment_name in names_to_plot:
        if experiment_name in data:
            # Plot the whole strain profile as a line
            plt.plot(data[experiment_name][:, 0], data[experiment_name][:, 1], 
                     marker=get_marker(experiment_name), label=experiment_name,
                     color=get_color(experiment_name, sample_sizes), linestyle=None)
        
            # Plot the moment before fracture as a larger point
            plt.plot(data[experiment_name][-1, 0], data[experiment_name][-1, 1], 
                     marker="o", color=get_color(experiment_name, sample_sizes), ms=10)
        
    plt.xlabel("Minor strain")
    plt.ylabel("Major strain")
    plt.title(title)
    plt.legend(loc='center right', bbox_to_anchor=(1.22, 0.5))
    plt.show()

strain_at_fracture = {}

for experiment_name, fracture_time in fracture_times.items():
    # Find the position of the maximum strain as it breaks. 
    max_strain_index = np.argmax(major_strain_data[experiment_name][-1])

    # Collect major and minor strain over time at the fracture position
    major_strain = [timestep[max_strain_index] for timestep in major_strain_data[experiment_name]]
    minor_strain = [timestep[max_strain_index] for timestep in minor_strain_data[experiment_name]]
    
    strain_at_fracture[experiment_name] = np.stack((minor_strain, major_strain), axis=-1)
    
experiment_names = list(fracture_times.keys())
interact(plot_strain_paths, names_to_plot=SelectMultiple(options=experiment_names, value=experiment_names), 
         data=fixed(strain_at_fracture), title=fixed("Strain paths for Surfalex to crack formation"));

We can see that the second repeat of the 60mm sample has failed. We will not use this data point for analysis. The rest of the strain profiles look reasonable so we take an average of the 3 runs as a point for the forming limit diagram later.

In [ ]:
# Remove the second repeat of 60 mm sample
fracture_times.pop("60mm_002", None)
strain_at_fracture.pop("60mm_002", None)

num_repeats = 3

def collect_final_strains(sample_sizes: List[str], fracture_times: dict, strain_at_fracture: Dict[str, np.ndarray]):
    fracture_strains = np.zeros((len(sample_sizes), 2))

    for index, sample_size in enumerate(sample_sizes):
        final_strain = np.zeros(2)
        num_samples = 0
        for repeat in range(1, num_repeats + 1):
            sample_name = f"{sample_size}_{repeat:03d}"
            if sample_name in fracture_times:
                final_strain += strain_at_fracture[sample_name][-1, :]
                num_samples += 1
        final_strain = final_strain / num_samples
        fracture_strains[index] = final_strain
    return fracture_strains

fracture_strains = collect_final_strains(sample_sizes, fracture_times, strain_at_fracture)

## 2). FLC from first derivative necking

First derivative necking determines the necking time to be when the strain in the sample becomes localised. This is determined numerically as the point at which the major strain in the necking region strongly exceeds the strain in the non-necking region. In this analysis, we set the cut-off point to be where the major strain in the necking region is 10 times the major strain in the non-necking region.

In order to measure this, we need to collect the evolution of the strain with time at the necking point and away from the necking point. In these samples the strain localisation is bimodal. We identify the necking point as the point of maximum strain at the timestep before the sample fails, this is typically at one end of the sample. We identify the point away from the neck to be the point of minimum strain between the two peaks at fracture.

In the below cells we build the analysis step by step.

The first cell shows strain profile at at fracture time. The maximum strain point is marked with a grey line and the minimum strain point is marked by a red line.

In [ ]:
def get_max_strain_index(major_strain_data: np.ndarray, experiment_name: str) -> int:
    """Finds the maximum strain for a strain profile of `experiment_name` at fracture time."""
    peaks, peak_properties = find_peaks(major_strain_data[experiment_name][-1], distance=50, height=0)
    return peaks[np.argmax(peak_properties["peak_heights"])]

def get_min_strain_index(major_strain_data: np.ndarray, experiment_name: str) -> int:
    """Finds the minimum strain between two maxima for a strain profile of `experiment_name` at fracture time."""
    peaks, peak_properties = find_peaks(major_strain_data[experiment_name][-1], distance=50, height=0)
    return np.argmin(major_strain_data[experiment_name][-1][peaks[0]:peaks[1]]) + peaks[0]

def plot_strain_max_min(major_strain_data: np.ndarray, experiment_name: str): 
    """Plot the identified minimum and maximum strain on the strain profile."""
    max_strain_index = get_max_strain_index(major_strain_data, experiment_name)
    min_strain_index = get_min_strain_index(major_strain_data, experiment_name)
    
    plt.figure(figsize=(10,6))
    plt.plot(major_strain_data[experiment_name][-1], label="Strain profile")
    plt.vlines(max_strain_index, plt.ylim()[0], plt.ylim()[1], alpha=0.5, color="k", label="Fracture point")
    plt.vlines(min_strain_index, plt.ylim()[0], plt.ylim()[1], color="r", alpha=0.5, label="Minimum strain")
    plt.title(f"Strain map of sample: {experiment_name} at fracture time")
    plt.xlabel("Vertical sample position")
    plt.ylabel("Strain")
    plt.legend()
    plt.show()

interact(plot_strain_max_min, major_strain_data=fixed(major_strain_data), 
         experiment_name=list(fracture_times.keys()));

We now collect the major and minor strain over time at the neck and away from the neck. 

We cut off the data from before 20 seconds since there are some early fluctuations which do not represent necking but make the later fitting harder. We know that the necking does not occur before 20 seconds in any of the samples.

In [ ]:
time_cutoff = 20

def get_index(value: float, data: Union[list, np.ndarray]) -> int:   
    """Return the index of the data point, closest to `value` in `data`."""
    return np.argmin(np.abs(np.array(data) - value))

def get_strain(experiment_name: str, frame_times: np.ndarray, major_strain_data: np.ndarray, crop_index: int):
    """Get strain at the neck and away from the neck. Crop values at inital value `crop_index`."""
    # Get indices of max and min strain
    max_strain_index = get_max_strain_index(major_strain_data, experiment_name)
    min_strain_index = get_min_strain_index(major_strain_data, experiment_name)

    # Collect strain over time at and away from neck
    major_strain_at_neck = [timestep[max_strain_index] for timestep in major_strain_data[experiment_name]]
    major_strain_away_neck = [timestep[min_strain_index] for timestep in major_strain_data[experiment_name]]
    
    return major_strain_at_neck[crop_index:], major_strain_away_neck[crop_index:]

def get_frame_times(experiment_name: str, crop_index: int) -> np.ndarray:
    """Get the time data for `experiment_name` starting at index `crop_index`"""
    return frame_times[experiment_name][crop_index:]

def plot_strain_over_time(experiment_name: str, frame_times: np.ndarray, major_strain_data: np.ndarray):
    # Get the index of the data point at `time_cutoff` seconds for cropping
    crop_index = get_index(frame_times[experiment_name], time_cutoff)
    
    time = get_frame_times(experiment_name, crop_index)
    major_strain_at_neck, major_strain_away_neck = get_strain(experiment_name, frame_times, major_strain_data, crop_index)

    plt.figure(figsize=(10, 6))

    plt.plot(time, major_strain_at_neck, "x", label="At neck")
    plt.plot(time, major_strain_away_neck, "x", label="Away from neck")
    plt.xlabel("Time (s)")
    plt.title(f"Major strain for experiment: {experiment_name} at two points")
    plt.ylabel("Strain")
    plt.legend()
    plt.show()

interact(plot_strain_over_time,
         experiment_name=list(fracture_times.keys()),
         frame_times=fixed(frame_times),
         major_strain_data=fixed(major_strain_data)
        );

Although the data looks good, it is a little noisy which becomes problematic when calculating the strain rate. For this reason we filter the strain rate with a median filter to smooth it. The kernel size of the median filter can be set in the below figure. Setting the size to 1 is equivilent to no smoothing.

The strain rate at the neck is fairly smooth, but the strain rate away form the neck is noisy because of the low values of strain. A kernel size of 13-15 is required to smooth the strain rate away from the neck.

In [ ]:
def plot_smooth_strain(experiment_name: str, filter_window: int, plot_neck: bool, plot_not_neck: bool):
    crop_index = get_index(frame_times[experiment_name], time_cutoff)
    
    time = get_frame_times(experiment_name, crop_index)
    major_strain_at_neck, major_strain_away_neck = get_strain(experiment_name, frame_times, major_strain_data, crop_index)

    if plot_neck:
        strain_rate_at_neck = np.gradient(major_strain_at_neck, time)
        smooth_strain_rate_at_neck = median_filter(strain_rate_at_neck, filter_window, mode="nearest")
        plt.plot(time, smooth_strain_rate_at_neck, "-", label="Smoothed neck strain rate")
    
    if plot_not_neck:
        strain_rate_away_neck = np.gradient(major_strain_away_neck, time)
        smooth_strain_rate_away_neck = median_filter(strain_rate_away_neck, filter_window, mode="nearest")
        plt.plot(time, smooth_strain_rate_away_neck, "-", label="Smoothed non-neck strain rate")
    
    plt.xlabel("Time (s)")
    plt.ylabel("Strain rate (/s)")
    plt.legend()
    plt.show()
    

interact(plot_smooth_strain,
         experiment_name=list(fracture_times.keys()),
         filter_window=IntSlider(min=1, value=1, max=20, description="Filter size.", continuous_update=False),
         plot_neck=Checkbox(value=True, description="Plot strain at neck"),
         plot_not_neck=Checkbox(value=True, description="Plot strain at non-neck region")
        );

The slider in the interactive graph allows you to find the time at which the sample reaches a certain strain ratio.

In [ ]:
def get_strain_ratio(experiment_name: str, frame_times: np.ndarray, major_strain_data: np.ndarray, crop_index: int, 
                     filter_window: int) -> np.ndarray:
    time = get_frame_times(experiment_name, crop_index)
    major_strain_at_neck, major_strain_away_neck = get_strain(experiment_name, frame_times, major_strain_data, crop_index)
    
    strain_rate_at_neck = np.gradient(major_strain_at_neck, time)
    strain_rate_away_neck = np.gradient(major_strain_away_neck, time)
    
    smooth_strain_rate_at_neck = median_filter(strain_rate_at_neck, filter_window, mode="nearest")
    smooth_strain_rate_away_neck = median_filter(strain_rate_away_neck, filter_window, mode="nearest")

    with np.errstate(divide='ignore'):
        ratio = smooth_strain_rate_at_neck / smooth_strain_rate_away_neck
    
    return ratio

def plot_strain_ratio(experiment_name: str, frame_times: np.ndarray, major_strain_data: np.ndarray, 
                      cutoff_ratio: int, filter_window: int):
    """Plot the ratio of the strain rate at the fracture point to the strain rate away from it."""
    crop_index = get_index(frame_times[experiment_name], time_cutoff)
    
    # X and Y data
    time = get_frame_times(experiment_name, crop_index)
    ratio = get_strain_ratio(experiment_name, frame_times, major_strain_data, crop_index, filter_window)
    # Find the time at which necking happens
    neck_index = get_index(ratio, cutoff_ratio)
    neck_time = time[neck_index] 

    plt.figure(figsize=(10, 6))
    plt.plot(time, ratio, "x")
    plt.vlines(neck_time, plt.ylim()[0], plt.ylim()[1])
    plt.hlines(cutoff_ratio, plt.xlim()[0], plt.xlim()[1])
    plt.title(f"Ratio of strain rate at neck to strain away\n from neck for experiment {experiment_name}")
    plt.xlabel("Time (s)")
    plt.ylabel("Ratio")
    plt.ylim(0, 20)
    plt.show()

interact(plot_strain_ratio,
         experiment_name=list(fracture_times.keys()),
         frame_times=fixed(frame_times),
         major_strain_data=fixed(major_strain_data),
         cutoff_ratio=IntSlider(min=1, value=10, max=20),
         filter_window=IntSlider(min=1, value=1, max=20, description="Filter size")
        );

We choose a ratio of 10 as the point at which the sample necks. We then use this as the cut off value for determining the forming limit curve.

In [ ]:
cutoff_ratio = 10
median_filter_window = 15

strain_at_necking = {}

for experiment_name, fracture_time in fracture_times.items():
    crop_index = get_index(frame_times[experiment_name], time_cutoff)
    
    time = get_frame_times(experiment_name, crop_index)
    ratio = get_strain_ratio(experiment_name, frame_times, major_strain_data, crop_index, median_filter_window)
    
    # The neck time is for the cropped data so have to add crop index back on to index the raw data
    neck_index = get_index(ratio, cutoff_ratio) + crop_index
    
    # Find the position of the maximum strain as sample breaks. 
    max_strain_index = np.argmax(major_strain_data[experiment_name][-1])
    
    # Collect strains over time at the break position
    major_strain = [timestep[max_strain_index] for timestep in major_strain_data[experiment_name]]
    minor_strain = [timestep[max_strain_index] for timestep in minor_strain_data[experiment_name]]

    strain_at_necking[experiment_name] = np.stack((minor_strain[:neck_index], major_strain[:neck_index]), axis=-1)
    
interact(plot_strain_paths, names_to_plot=SelectMultiple(options=experiment_names, value=experiment_names), 
         data=fixed(strain_at_necking), title=fixed("Strain paths for Surfalex to necking"));

In [ ]:
necking_strains = collect_final_strains(sample_sizes, fracture_times, strain_at_necking)

# Plot FLD

We can now plot the experimental data against the data measured by Constellium

In [ ]:
plt.figure()

# Plot strain at fracture
plt.plot(fracture_strains[:, 0], fracture_strains[:, 1], 'o', label="Strain at fracture")

plt.plot(necking_strains[:, 0], necking_strains[:, 1], 'x', label="Strain at necking")

# Plot constellium strain
data = np.loadtxt(data_folder / 'constellium/constellium_data.txt')
plt.plot(data[:, 0], data[:, 1], label="Surfalex")
plt.plot(data[:, 2], data[:, 3], label="Surfalex HF")

# Make plot pretty
plt.ylim(0.180, 0.46)
plt.xlim(-0.2, 0.4)
plt.ylabel('Major strain')
plt.xlabel('Minor strain')
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
ax.grid(axis='y', alpha=0.5)
    
plt.legend()
plt.savefig(RESULTS_DIR / "FLD.png", dpi=200)